In [1]:
path_root = r"C:/Users/jgpg000.edu/Desktop/Agente inteligente recuperación sensórica IoT/"

# ruta clases personalizadas
import sys
path_classes = f"{path_root}classes"
sys.path.append(path_classes)

## LIBRERIAS
import pandas as pd
from class_data import *
from class_dataset import *
from fun_models import *

## Agente inteligente

#### Cargar base de datos

In [2]:
path_excel = f"{path_root}datasets/database_items/item_list.xlsx"
path_saved_json = f"{path_root}datasets/database_items/database_json/"

documentos = load_documents(path_excel, path_saved_json)
documentos_langchain = load_documents_langchain(documentos)

from langchain_text_splitters import RecursiveCharacterTextSplitter
# Aplicamos el splitter de Langchain
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

# Dividimos cada texto en chunks
chunks_texto = text_splitter.split_documents(documentos_langchain)
print(f"Hay {len(chunks_texto)} chunks de texto.")

100%|██████████| 88/88 [00:00<00:00, 319.01it/s]


Se han cargado 86 documentos.


100%|██████████| 86/86 [00:00<00:00, 9702.51it/s]


Se han generado 954 documentos de Langchain.
    De ellos, 86 son el texto y 868 son resúmenes de tablas.
Hay 7218 chunks de texto.


#### Modelo RAG

In [3]:
path_saved_vector = f"{path_root}datasets/database_items/database_vectorial/"
name_database = "large_esp"
modelo_llm = "gpt-4o"

template_sistema = """ Eres un asistente de una empresa encargada de smart cities.
Como preguntas, te van a adjuntar listas de requisitos de equipo para proyecto.
Tu labor es responder con el nombre de los equipos que encuentres que cumplan los requisitos, así como suministrar la información de estos dispositivos en relación a los requisitos.
Usa la información añadida en el contexto para responder a la pregunta pero no hagas referencias al mismo.
Se añade como metadato tanto el proveedor como el nombre a los que pertenece el contexto devuelto.
Responde únicamente con la especificación que se pide. Si no aparece di que no lo sabes.
Pregunta: {question}
Metadato: {metadata}
Contexto: {contexts}
La respuesta debe tener el siguiente formato:
- Nombre del equipo y proveedor.
- Especificaciones que se han preguntado y aparecen en el contexto.
Respuesta:
"""
# Creamos la vectorstore
vectorstore = create_database(f"{path_saved_vector}{name_database}",
                               chunks_texto, nombre_embedding = "text-embedding-3-large")

Definimos el modelo como una clase para que mantenga el formato de llamadas con .invoke

```from class_rag_mode import mode_item_global```

#### Ejemplo de ejecución

In [4]:
pregunta_ejemplo = '''Se requiere de un sonómetro clase II LoRa que disponga de  las siguientes especificaciones como mínimo:
- Rango: 40 dB a 115 dB. 
- Frecuencia de ponderación A.
- Capacidad para proveer diferentes ponderaciones para el correcto estudio del ruido: LAeq, 
LA1, LA10, LA50, LA90, LA99, LAmax, LAmin, LA Slow o LA Fast entre otros.
- Detección de picos.
- Consumo: 40 mA.
- Protector contra viento.
- Voltaje de operación: 3.6 a 6V.
- Normativa IEC 61672-1.
- Certificación: ROHS / CE.
'''

In [5]:
from class_rag_mode import rag_mode_global
productos, proveedores = rag_mode_global.mode_item_global(vectorstore,
                        pregunta = pregunta_ejemplo,
                        k=3,
                        num_productos_maximo=3,
                        tipo_equipo="sonometro",
                        path_item_list = path_excel)
print(productos,": ", proveedores)

[('B&K 2245', 15), ('Smart Spot Sound level Meter clase II', 8), ('Clase 1 modelo TA120', 7)] :  [('HBK', 15), ('Hopu', 8), ('CESVA', 7)]


In [6]:
from class_rag_mode import rag_mode_global
RAG_mode = rag_mode_global(vectorstore, verbose=True)
response = RAG_mode.invoke(pregunta_ejemplo,
                        tipo_equipo="sonometro",
                        k=3, # Productos extraidos por sentencia
                        num_productos_maximo=3)
response['answer']

RAG creado correctamente.
        Llámalo con <rag_name>.invoke(<query>, tipo_equipo, k, num_productos_maximo)
Productos elegidos: ['B&K 2245', 'Smart Spot Sound level Meter clase II', 'Clase 1 modelo TA120']


'- Nombre del equipo y proveedor: Smart Spot Sound level Meter clase II, Hopu.\n- Especificaciones que se han preguntado y aparecen en el contexto:\n  - Rango: 40 dB a 115 dB.\n  - Frecuencia de ponderación A.\n  - Capacidad para proveer diferentes ponderaciones para el correcto estudio del ruido: LAeq, LA1, LA10, LA50, LA90, LA99, LAmax, LAmin, LA Slow, LA Fast.\n  - Detección de picos.\n  - Normativa IEC 61672-1.\n  - Certificación: ROHS / CE.'

#### Interfaz gráfica

In [ ]:
import gradio as gr

def predict(message, history):
    response = RAG_mode.invoke(message)
    return response['answer']

gr.ChatInterface(predict).launch()